In [9]:
import numpy as np

size = 1
variance = 0.5
x, y = np.mgrid[-size:size+1, -size:size+1]
print(x)
g = np.exp(-(x**2 + y**2)/float(2*variance))

print(g)

[[-1 -1 -1]
 [ 0  0  0]
 [ 1  1  1]]
[[0.13533528 0.36787944 0.13533528]
 [0.36787944 1.         0.36787944]
 [0.13533528 0.36787944 0.13533528]]


In [27]:
def gaussian_kernel(size, variance):
    x, y = np.mgrid[-size:size+1, -size:size+1]
    g = 255 * np.exp(-(x**2 + y**2)/float(2*variance))
    return g.astype(int)

In [29]:
print(gaussian_kernel(1, 0.5))

[[ 34  93  34]
 [ 93 255  93]
 [ 34  93  34]]


In [45]:
import pandas as pd
labels = pd.read_csv('datasets/TrackNet/game1/Clip1/Label.csv')
print(labels)

    file name  visibility  x-coordinate  y-coordinate  status
0    0000.jpg           1         599.0         423.0     0.0
1    0001.jpg           1         601.0         406.0     0.0
2    0002.jpg           1         601.0         388.0     0.0
3    0003.jpg           1         598.0         371.0     0.0
4    0004.jpg           1         598.0         353.0     0.0
..        ...         ...           ...           ...     ...
202  0202.jpg           1        1171.0         553.0     0.0
203  0203.jpg           1        1198.0         577.0     0.0
204  0204.jpg           1        1230.0         607.0     0.0
205  0205.jpg           1        1261.0         639.0     0.0
206  0206.jpg           1        1279.0         661.0     0.0

[207 rows x 5 columns]


In [42]:
import os
import pandas as pd
import cv2


def create_ground_truth_images(input_path, output_path, size, variance, height, width):
    gaussian_kernel_array = gaussian_kernel(size, variance)
    for game_id in range(1, 11):
        game = 'game{}'.format(game_id)
        clips = os.listdir(os.path.join(input_path, game))
        for clip in clips:
            print('game: {}, clip: {}'.format(game, clip))

            # Create the directory structure for game
            path_out_game = os.path.join(output_path, game)
            if not os.path.exists(path_out_game):
                os.makedirs(path_out_game)

            # Create the directory structure for clip
            path_out_clip = os.path.join(path_out_game, clip)
            if not os.path.exists(path_out_clip):
                os.makedirs(path_out_clip)

            # Load the labels
            path_labels = os.path.join(input_path, game, clip, 'Label.csv')
            labels = pd.read_csv(path_labels)
            for idx in range(labels.shape[0]):
                # Extract the label information
                file_name, visibility, x, y, status = labels.iloc[idx]
                # Create the heatmap
                heatmap = np.zeros((height, width), dtype=np.uint8)

                if visibility != 0:
                    x = int(x)
                    y = int(y)
                    for i in range(-size, size+1):
                        for j in range(-size, size+1):
                            if x + i >= 0 and x + i < width and y + j >= 0 and y + j < height:
                                heatmap[y + j, x + i] = gaussian_kernel_array[j + size, i + size]

                cv2.imwrite(os.path.join(path_out_clip, file_name), heatmap)



In [55]:
def create_ground_truth_labels(input_path, output_path, train_rate=0.7):
    df = pd.DataFrame()
    for game_id in range(1, 11):
        game = 'game{}'.format(game_id)
        clips = os.listdir(os.path.join(input_path, game))
        for clip in clips:
            # Load the labels
            path_labels = os.path.join(input_path, game, clip, 'Label.csv')
            labels = pd.read_csv(path_labels)
            labels['img_path_1'] = 'images/' + game +'/' + clip + '/' + labels['file name']
            labels['ground_truth_path'] = 'ground_truth/' + game +'/' + clip + '/' + labels['file name']
            
            # We startat the third image to have 3 consecutive images
            labels_target = labels[2:].copy()
            labels_target.loc[:, 'img_path_2'] = list(labels['img_path_1'][1:-1])
            labels_target.loc[:, 'img_path_3'] = list(labels['img_path_1'][:-2])
            df = pd.concat([df, labels_target])
    
    df = df.reset_index(drop=True)
    df = df[['img_path_1', 'img_path_2', 'img_path_3', 'ground_truth_path', 'x-coordinate', 'y-coordinate', 'visibility', 'status']]
    
    # We shuffle the data
    df = df.sample(frac=1)
    num_train = int(train_rate * df.shape[0])
    df_train = df[:num_train]
    df_test = df[num_train:]
    df_train.to_csv(os.path.join(output_path, 'labels_train.csv'), index=False)
    df_test.to_csv(os.path.join(output_path, 'labels_test.csv'), index=False)
    print(df)
    
create_ground_truth_labels('datasets/TrackNet', 'datasets/TrackNet')      

                         img_path_1                    img_path_2  \
3956    images/game2/Clip8/0443.jpg   images/game2/Clip8/0442.jpg   
10548   images/game6/Clip1/0364.jpg   images/game6/Clip1/0363.jpg   
10406   images/game6/Clip1/0222.jpg   images/game6/Clip1/0221.jpg   
6425    images/game4/Clip1/0399.jpg   images/game4/Clip1/0398.jpg   
17421   images/game9/Clip9/0119.jpg   images/game9/Clip9/0118.jpg   
...                             ...                           ...   
478    images/game1/Clip12/0040.jpg  images/game1/Clip12/0039.jpg   
3530    images/game2/Clip8/0017.jpg   images/game2/Clip8/0016.jpg   
15851   images/game8/Clip9/0119.jpg   images/game8/Clip9/0118.jpg   
11772   images/game6/Clip3/0417.jpg   images/game6/Clip3/0416.jpg   
2726    images/game2/Clip5/0143.jpg   images/game2/Clip5/0142.jpg   

                         img_path_3                   ground_truth_path  \
3956    images/game2/Clip8/0441.jpg   ground_truth/game2/Clip8/0443.jpg   
10548   images/game6/